In [1]:
import pyanitools as pyt
import pyaniasetools as aat
import hdnntools as hdt
import numpy as np

import matplotlib.pyplot as plt
import aniensemblestats as ens

import os
import pandas as pd

import time

import seaborn as sns
print(pd.__version__)

#import seaborn as sns
pd.options.display.float_format = '{:.2f}'.format
cm = sns.light_palette("green", as_cmap=True)

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0.20.3


In [2]:
tex_store = '/home/jsmith48/MDBenchmark_ANI-9.0.5.tex'
xls_store = '/home/jsmith48/MDBenchmark_ANI-9.0.5.xlsx'
frc_xlsx = '/home/jsmith48/MDBenchmark_frcErrorperMolecule_ANI-9.0.5.xlsx'
ani_version = 'ANI-9.0.5'

fdir_wb97 = '/home/jsmith48/scratch/extensibility_test_sets/COMP6v1/ani_md_benchmark/data_big/data/'
fdir_dftb = '/home/jsmith48/scratch/extensibility_test_sets/COMP6v1/ani_md_benchmark/test_dftb/data/'
fdir_pm6 = '/home/jsmith48/scratch/extensibility_test_sets/COMP6v1/ani_md_benchmark/test_pm6/data/'

wkdir = '/home/jsmith48/scratch/ANI-2x_retrain/train/ANI-2x_v62318/'
#wkdir = '/home/jujuman/Research/ANI-validation/'
cnstfile = wkdir + 'train0/rHCNOSFCl-5.2R_16-3.8A_a4-8.params'
saefile  = wkdir + 'train0/sae_linfit.dat'
nnfdir   = wkdir + '/train'
Nn = 8

In [3]:
# Define the conformer cross validator class
anicv = aat.anicrossvalidationmolecule(cnstfile,saefile,nnfdir,Nn,gpuid=6)

In [4]:
files = os.listdir(fdir_dftb)

stats1 = dict()
stats2 = dict()
stats3 = dict()
stats4 = dict()
style_F = dict()
Forces = dict({'ANI':[],
               'DFT':[],
               'DFTB':[],
               'PM6':[],})

forces = dict({'ANI':{},
               'DFT':{},
               'DFTB':{},
               'PM6':{},})


denegy = dict({'ANI':{},
               'DFT':{},
               'DFTB':{},
               'PM6':{},})

enegy = dict({'ANI':{},
               'DFT':{},
               'DFTB':{},
               'PM6':{},})

for f in files:
    name = f.split('_')[0]
    data_wb97 = hdt.readncdatall(fdir_wb97+f)
    data_dftb = hdt.readncdatall(fdir_dftb+f)
    data_pm6 = hdt.readncdatall(fdir_pm6+f)
    
    
    start = time.time()
    Eani, Fani, Sani = anicv.compute_stats_multi_molecule(X=data_wb97['coordinates'],S=data_wb97['species'])
    end = time.time()
    comp_time = end - start
    
    Sani = np.std(Eani,axis=0)
    print(Sani.shape)
    Eani = np.mean(Eani,axis=0)
    Fani = -np.mean(Fani,axis=0)
    
    Ewb97 = hdt.hatokcal*data_wb97['energies']
    Fwb97 = hdt.hatokcal*data_wb97['forces']
    
    Edftb = hdt.hatokcal*data_dftb['energies']
    Fdftb = hdt.hatokcal*data_dftb['forces']

    Epm6 = hdt.hatokcal*data_pm6['energies']
    Fpm6 = hdt.hatokcal*data_pm6['forces']
    
    print(name,Ewb97.size,' Time: ', comp_time)
    #print(data['species'])
    #print((Fani-Fdft)[0][12])
    #print(Fani[0][12])
    #print(Fdft[0][12])

    frc_error = dict()
    dF = np.linalg.norm(Fani - Fwb97,axis=2)
    #dF = np.mean(dF,axis=1)
    print(dF.shape)
    for j,F in enumerate(dF):
        Fstore = dict()
        for i,(s,x) in enumerate(zip(data_wb97['species'],F)):
            Fstore[str(i).zfill(3)+' '+s] = x
        #Fstore['Total'] = np.mean(dF,axis=1)
        #print(np.mean(dF,axis=1).shape)
        frc_error[str(j).zfill(3)] = pd.Series(Fstore)
    style_F[name] = pd.DataFrame(frc_error).transpose().style.background_gradient(cmap=cm)
    
    #np.random.shuffle(Fwb97)
    
    Fani = Fani.flatten()
    Fwb97 = Fwb97.flatten()
    Fdftb = Fdftb.flatten()
    Fpm6 = Fpm6.flatten()
    
    Forces['ANI'].append(Fani)
    Forces['DFT'].append(Fwb97)
    Forces['DFTB'].append(Fdftb)
    Forces['PM6'].append(Fpm6)

    forces['ANI'].update({name : Fani})
    forces['DFT'].update({name : Fwb97})
    forces['DFTB'].update({name : Fdftb})
    forces['PM6'].update({name : Fpm6})
    
    dEani = hdt.calculatedmat(Eani)
    dEwb97 = hdt.calculatedmat(Ewb97)
    dEdftb = hdt.calculatedmat(Edftb)
    dEpm6 = hdt.calculatedmat(Epm6)
    
    denegy['ANI'].update({name : dEani})
    denegy['DFT'].update({name : dEwb97})
    denegy['DFTB'].update({name : dEdftb})
    denegy['PM6'].update({name : dEpm6})
    
    enegy['ANI'].update({name : Eani})
    enegy['DFT'].update({name : Ewb97})
    enegy['DFTB'].update({name : Edftb})
    enegy['PM6'].update({name : Epm6})
    
    Na = len(data_wb97['species'])
    
    stats1[name] = pd.Series({'$\mathrm{Atoms}$': int(Na),
                              '$\mathrm{E_{MAE}}$'   : hdt.calculatemeanabserror(Eani,Ewb97),
                              '$\mathrm{E_{RMS}}$'   : hdt.calculaterootmeansqrerror(Eani,Ewb97),
                              '$\mathrm{E_{range}}$'   : np.abs(Ewb97.max()-Ewb97.min()),
                              '$\mathrm{\Delta E_{MAE}}$'  : hdt.calculatemeanabserror(dEani,dEwb97),
                              '$\mathrm{\Delta E_{RMS}}$'  : hdt.calculaterootmeansqrerror(dEani,dEwb97),
                              '$\mathrm{\Delta E_{range}}$'   : np.abs(dEwb97.max()-dEwb97.min()),
                              '$\mathrm{F_{MAE}}$'   : hdt.calculatemeanabserror(Fani,Fwb97),
                              '$\mathrm{F_{RMS}}$'   : hdt.calculaterootmeansqrerror(Fani,Fwb97),
                              '$\mathrm{F_{range}}$'   : np.abs(Fwb97.max()-Fwb97.min()),
                              'time(ms)/mol' : 1000*comp_time/Ewb97.size,
                              })
    
    stats2[name] = pd.Series({'$\mathrm{Atoms}$': int(Na),
                              '$\mathrm{\Delta E_{MAE}^{DFTB}}$' : hdt.calculatemeanabserror(dEdftb,dEwb97),
                              '$\mathrm{\Delta E_{MAE}^{PM6}}$'  : hdt.calculatemeanabserror(dEpm6,dEwb97),
                              '$\mathrm{\Delta E_{MAE}^{ANI}}$'  : hdt.calculatemeanabserror(dEani,dEwb97),
                              '$\mathrm{\Delta E_{RMS}^{DFTB}}$' : hdt.calculaterootmeansqrerror(dEdftb,dEwb97),
                              '$\mathrm{\Delta E_{RMS}^{PM6}}$'  : hdt.calculaterootmeansqrerror(dEpm6,dEwb97),
                              '$\mathrm{\Delta E_{RMS}^{ANI}}$'  : hdt.calculaterootmeansqrerror(dEani,dEwb97),
                              '$\mathrm{F_{MAE}^{DFTB}}$'   : hdt.calculatemeanabserror(Fdftb,Fwb97),
                              '$\mathrm{F_{MAE}^{PM6}}$'   : hdt.calculatemeanabserror(Fpm6,Fwb97),
                              '$\mathrm{F_{MAE}^{ANI}}$'   : hdt.calculatemeanabserror(Fani,Fwb97),
                              '$\mathrm{F_{RMS}^{DFTB}}$'   : hdt.calculaterootmeansqrerror(Fdftb,Fwb97),
                              '$\mathrm{F_{RMS}^{PM6}}$'   : hdt.calculaterootmeansqrerror(Fpm6,Fwb97),
                              '$\mathrm{F_{RMS}^{ANI}}$'   : hdt.calculaterootmeansqrerror(Fani,Fwb97),
                              })
 
    stats4[name] = pd.Series({'$\mathrm{Atoms}$': int(Na),
                              '$\mathrm{E_{RMS}/ atom}$'   : hdt.calculaterootmeansqrerror(Eani/Na,Ewb97/Na),
                              '$\mathrm{E_{RMS}/ \sqrt{atom}}$'   : hdt.calculaterootmeansqrerror(Eani,Ewb97)/np.sqrt(Na),
                              '$\mathrm{\sigma / \sqrt{atom}}$'   : np.mean(Sani/np.sqrt(Na)),
                              '$\mathrm{\Delta E_{RMS}/ atom}$'  : hdt.calculaterootmeansqrerror(dEani,dEwb97)/Na,
                              '$\mathrm{\Delta E_{RMS}/ \sqrt{atom}}$'  : hdt.calculaterootmeansqrerror(dEani,dEwb97)/np.sqrt(Na),
                              '$\mathrm{F_{RMS}/ atom}$'   : hdt.calculaterootmeansqrerror(Fani,Fwb97)/Na,
                              '$\mathrm{F_{RMS}/ \sqrt{atom}}$'   : hdt.calculaterootmeansqrerror(Fani,Fwb97)/np.sqrt(Na),
                              })

    #for i,E in enumerate(Eani-Ewb97):
    #    print(' -',i,E)

    dErANI_MAE = hdt.calculatemeanabserror(dEani,dEwb97)
    dErDFTB_MAE = hdt.calculatemeanabserror(dEdftb,dEwb97)
    dErPM6_MAE = hdt.calculatemeanabserror(dEpm6,dEwb97)

    dErANI_RMS = hdt.calculaterootmeansqrerror(dEani,dEwb97)
    dErDFTB_RMS = hdt.calculaterootmeansqrerror(dEdftb,dEwb97)
    dErPM6_RMS = hdt.calculaterootmeansqrerror(dEpm6,dEwb97)
    
    FrANI_MAE = hdt.calculatemeanabserror(Fani,Fwb97)
    FrDFTB_MAE = hdt.calculatemeanabserror(Fdftb,Fwb97)
    FrPM6_MAE = hdt.calculatemeanabserror(Fpm6,Fwb97)

    FrANI_RMS = hdt.calculaterootmeansqrerror(Fani,Fwb97)
    FrDFTB_RMS = hdt.calculaterootmeansqrerror(Fdftb,Fwb97)
    FrPM6_RMS = hdt.calculaterootmeansqrerror(Fpm6,Fwb97)
    
    stats3[name] = pd.Series({#'$\mathrm{Atoms}$': str(int(Na)),
                              '$\mathrm{\Delta \Delta E_{MAE}^{DFTB}}$'  : dErDFTB_MAE-dErANI_MAE,
                              '$\mathrm{\Delta \Delta E_{MAE}^{PM6}}$'  : dErPM6_MAE-dErANI_MAE,
                              '$\mathrm{\Delta \Delta E_{RMS}^{DFTB}}$'  : dErDFTB_RMS-dErANI_RMS,
                              '$\mathrm{\Delta \Delta E_{RMS}^{PM6}}$'  : dErPM6_RMS-dErANI_RMS,
                              '$\mathrm{\Delta F_{MAE}^{DFTB}}$'   : FrDFTB_MAE-FrANI_MAE,
                              '$\mathrm{\Delta F_{MAE}^{PM6}}$'   : FrPM6_MAE-FrANI_MAE,
                              '$\mathrm{\Delta F_{RMS}^{DFTB}}$'   : FrDFTB_RMS-FrANI_RMS,
                              '$\mathrm{\Delta F_{RMS}^{PM6}}$'   : FrPM6_RMS-FrANI_RMS,
                              })

for k in Forces.keys():
    Forces[k] = np.concatenate(Forces[k])
    
edat1 = pd.DataFrame(stats1)
edat2 = pd.DataFrame(stats2)
edat3 = pd.DataFrame(stats3)
edat4 = pd.DataFrame(stats4)

edat4 = edat4.transpose()
edat4 = edat4.sort_values(['$\mathrm{Atoms}$'])
edat4 = edat4.transpose()

edat2['mean'] = edat2.mean(axis=1)
edat4['mean'] = edat4.mean(axis=1)

print(edat2['mean'])

edat1 = edat1.transpose()
edat2 = edat2.transpose()
edat3 = edat3.transpose()
edat4 = edat4.transpose()

(128,)
Atazanavir 128  Time:  3.242431163787842
(128, 103)
(128,)
ChignolinNeutral 128  Time:  4.02389931678772
(128, 149)
(128,)
Caffeine 128  Time:  1.686654806137085
(128, 24)
(127,)
Fentanyl 127  Time:  2.8580360412597656
(127, 53)
(128,)
Salbutamol 128  Time:  2.4451382160186768
(128, 38)
(128,)
Tolterodine 128  Time:  3.2996935844421387
(128, 55)
(128,)
Lisdexamfetamine 128  Time:  2.204869270324707
(128, 44)
(128,)
Oseltamivir 128  Time:  2.7061150074005127
(128, 50)
(128,)
Capsaicin 128  Time:  2.0582821369171143
(128, 49)
(128,)
Ranolazine 128  Time:  2.5450751781463623
(128, 64)
(128,)
Atomoxetine 128  Time:  2.3244121074676514
(128, 40)
(128,)
Retinol 128  Time:  2.685922622680664
(128, 51)
(128,)
Acetaminophen 128  Time:  1.3965091705322266
(128, 20)
(128,)
TrpCage1L2Y 128  Time:  4.3650805950164795
(128, 312)
$\mathrm{Atoms}$                   75.14
$\mathrm{F_{MAE}^{ANI}}$            2.32
$\mathrm{F_{MAE}^{DFTB}}$           4.51
$\mathrm{F_{MAE}^{PM6}}$            7.63
$\

In [5]:
#edat = pd.DataFrame(stats).transpose()
pd.set_option('expand_frame_repr', False)
edat1 = edat1.sort_values(['$\mathrm{Atoms}$'])
#edat2 = edat2.sort_values(['$\mathrm{Atoms}$'])
#edat3 = edat3.sort_values(['$\mathrm{Atoms}$'])

# Build latex document
header = r'\documentclass{article}'+'\n'+r'\usepackage{booktabs}'+'\n'+r'\usepackage[margin=0.5in]{geometry}'+'\n'+r'\begin{document}'+'\n'

begtab = r'\begin{table}'+'\n'+r'\tiny'+'\n'+r'\centering'+'\n'
endtab = r'\end{table}'+'\n'
enddoc = r'\end{document}'
with open(tex_store, "w") as text_file:
    text_file.write(header)
    text_file.write(begtab+r'\caption{'+ani_version+r'}'+'\n'+edat1.to_latex(escape=False)+endtab)
    text_file.write(begtab+r'\caption{'+'ANI per atom errors'+r'}'+'\n'+edat4.to_latex(escape=False)+endtab)
    text_file.write(begtab+r'\caption{'+'ANI vs. DFTB vs. PM6'+r'}'+'\n'+edat2.to_latex(escape=False)+endtab)
    text_file.write(begtab+r'\caption{'+'\|DFTB - '+ani_version+'\| and \|PM6 -'+ani_version+r'|}'+'\n'+edat3.to_latex(escape=False)+endtab)
    text_file.write(enddoc)
    

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(xls_store, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
edat1.to_excel(writer, sheet_name=ani_version)
edat2.to_excel(writer, sheet_name='DFTB')
edat3.to_excel(writer, sheet_name='DFTB vs. ANI')
edat4.to_excel(writer, sheet_name='Per atom errors')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [6]:
edat1.style

,$\mathrm{Atoms}$,$\mathrm{E_{MAE}}$,$\mathrm{E_{RMS}}$,$\mathrm{E_{range}}$,$\mathrm{F_{MAE}}$,$\mathrm{F_{RMS}}$,$\mathrm{F_{range}}$,$\mathrm{\Delta E_{MAE}}$,$\mathrm{\Delta E_{RMS}}$,$\mathrm{\Delta E_{range}}$,time(ms)/mol
Acetaminophen,20,0.885874,1.00374,15.6718,1.65757,2.33689,196.649,0.59433,0.751993,30.1989,10.9102
Caffeine,24,0.869366,1.12825,17.0612,2.67671,4.07298,260.109,1.12292,1.4487,34.1183,13.177
Salbutamol,38,1.56184,1.87512,21.1453,2.05622,2.82766,226.81,1.22956,1.53706,42.0731,19.1026
Atomoxetine,40,0.912265,1.14188,19.6671,1.87651,2.69149,203.001,1.27315,1.61957,36.4957,18.1595
Lisdexamfetamine,44,0.6915,0.890921,27.9031,1.65263,2.287,229.519,0.994101,1.26117,54.4246,17.2255
Capsaicin,49,1.22705,1.59825,25.7911,1.92263,2.7476,231.132,1.7674,2.26046,50.2585,16.0803
Oseltamivir,50,1.38149,1.88716,40.7182,2.3491,3.57416,233.45,1.94938,2.5211,79.6821,21.1415
Retinol,51,7.49224,8.14746,40.9504,3.7768,7.34601,285.628,3.54748,4.57534,75.4607,20.9838
Fentanyl,53,1.3521,1.69019,29.7843,1.99466,2.8691,208.104,1.46293,1.85225,55.2531,22.5042
Tolterodine,55,1.3706,1.66385,28.2822,2.08685,2.97442,205.093,1.7352,2.14306,54.4357,25.7789


In [7]:
edat2

,$\mathrm{Atoms}$,$\mathrm{F_{MAE}^{ANI}}$,$\mathrm{F_{MAE}^{DFTB}}$,$\mathrm{F_{MAE}^{PM6}}$,$\mathrm{F_{RMS}^{ANI}}$,$\mathrm{F_{RMS}^{DFTB}}$,$\mathrm{F_{RMS}^{PM6}}$,$\mathrm{\Delta E_{MAE}^{ANI}}$,$\mathrm{\Delta E_{MAE}^{DFTB}}$,$\mathrm{\Delta E_{MAE}^{PM6}}$,$\mathrm{\Delta E_{RMS}^{ANI}}$,$\mathrm{\Delta E_{RMS}^{DFTB}}$,$\mathrm{\Delta E_{RMS}^{PM6}}$
Acetaminophen,20.00,1.66,4.87,7.23,2.34,7.16,10.82,0.59,1.59,2.23,0.75,2.00,2.80
Atazanavir,103.00,2.32,5.11,8.14,3.40,7.65,12.06,3.06,4.63,7.43,3.78,5.71,9.27
Atomoxetine,40.00,1.88,3.23,6.38,2.69,4.57,9.25,1.27,2.41,3.12,1.62,3.02,3.85
Caffeine,24.00,2.68,7.30,12.02,4.07,12.07,19.79,1.12,2.33,4.92,1.45,2.93,6.08
Capsaicin,49.00,1.92,3.58,6.72,2.75,5.33,9.86,1.77,2.43,3.76,2.26,3.02,4.67
ChignolinNeutral,149.00,2.97,5.87,9.76,4.25,8.44,13.65,3.84,5.50,9.62,4.80,7.05,12.10
Fentanyl,53.00,1.99,3.46,6.71,2.87,5.12,9.60,1.46,1.98,3.81,1.85,2.47,4.76
Lisdexamfetamine,44.00,1.65,3.53,6.84,2.29,5.26,9.89,0.99,2.27,4.24,1.26,2.84,5.29
Oseltamivir,50.00,2.35,4.56,7.31,3.57,6.79,10.41,1.95,2.76,4.00,2.52,3.47,4.94
Ranolazine,64.00,2.11,4.20,8.26,2.96,5.92,11.66,1.78,4.01,6.03,2.29,4.97,7.49


In [8]:
s = edat3.style.background_gradient(cmap=cm)
s

,$\mathrm{\Delta F_{MAE}^{DFTB}}$,$\mathrm{\Delta F_{MAE}^{PM6}}$,$\mathrm{\Delta F_{RMS}^{DFTB}}$,$\mathrm{\Delta F_{RMS}^{PM6}}$,$\mathrm{\Delta \Delta E_{MAE}^{DFTB}}$,$\mathrm{\Delta \Delta E_{MAE}^{PM6}}$,$\mathrm{\Delta \Delta E_{RMS}^{DFTB}}$,$\mathrm{\Delta \Delta E_{RMS}^{PM6}}$
Acetaminophen,3.21001,5.5723,4.82258,8.48332,1.00033,1.63966,1.25023,2.04792
Atazanavir,2.78524,5.81388,4.24734,8.66106,1.56982,4.37185,1.93042,5.49073
Atomoxetine,1.35806,4.50043,1.87666,6.55722,1.13593,1.85031,1.39606,2.22672
Caffeine,4.62289,9.3461,7.99221,15.7134,1.21045,3.79398,1.4839,4.62848
Capsaicin,1.65705,4.80035,2.58234,7.1114,0.662169,1.99437,0.763109,2.41355
ChignolinNeutral,2.89489,6.78465,4.18563,9.40001,1.66522,5.77738,2.25447,7.30249
Fentanyl,1.46511,4.7184,2.25383,6.73203,0.51241,2.35125,0.615282,2.90369
Lisdexamfetamine,1.87243,5.18627,2.97774,7.5987,1.28047,3.24224,1.58191,4.03019
Oseltamivir,2.21478,4.95599,3.21716,6.83324,0.813607,2.05173,0.952425,2.42068
Ranolazine,2.09302,6.15364,2.96352,8.69811,2.22393,4.25172,2.68299,5.20593


In [9]:
edat4.style

,$\mathrm{Atoms}$,$\mathrm{E_{RMS}/ \sqrt{atom}}$,$\mathrm{E_{RMS}/ atom}$,$\mathrm{F_{RMS}/ \sqrt{atom}}$,$\mathrm{F_{RMS}/ atom}$,$\mathrm{\Delta E_{RMS}/ \sqrt{atom}}$,$\mathrm{\Delta E_{RMS}/ atom}$,$\mathrm{\sigma / \sqrt{atom}}$
Acetaminophen,20,0.224442,0.0501868,0.522545,0.116845,0.168151,0.0375996,0.143749
Caffeine,24,0.230304,0.0470106,0.831393,0.169707,0.295714,0.0603624,0.368158
Salbutamol,38,0.304184,0.0493452,0.458707,0.0744122,0.249345,0.040449,0.219793
Atomoxetine,40,0.180547,0.0285469,0.425562,0.0672873,0.256076,0.0404892,0.221752
Lisdexamfetamine,44,0.134311,0.0202482,0.344779,0.0519774,0.190129,0.0286631,0.12462
Capsaicin,49,0.228321,0.0326173,0.392515,0.0560735,0.322923,0.0461319,0.20112
Oseltamivir,50,0.266885,0.0377432,0.505463,0.0714833,0.356537,0.050422,0.202696
Retinol,51,1.14087,0.159754,1.02865,0.144039,0.640675,0.0897125,0.301681
Fentanyl,53,0.232165,0.0318903,0.394102,0.054134,0.254426,0.0349481,0.265837
Tolterodine,55,0.224354,0.0302519,0.40107,0.0540803,0.28897,0.0389647,0.242244


In [10]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(frc_xlsx, engine='xlsxwriter')


for k in style_F:
    # Convert the dataframe to an XlsxWriter Excel object.
    style_F[k].to_excel(writer, sheet_name=k+'_'+ani_version)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [11]:
import aniensemblestats as aes
import matplotlib as mpl

from matplotlib.backends.backend_pdf import PdfPages

In [12]:
storepath = '/home/jujuman/Dropbox/ANAKIN-ME-RESEARCH/PUBLICATION-ActiveLearning/Figures/force_ANIMD_compare_complete.pdf'

label_size = 24
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size

#'$F_{dft}$' + r' $(kcal \times mol^{-1} \times \AA^{-1})$'

fig, axes = plt.subplots(1, 3, figsize=[30,8], sharex=True, sharey=True, gridspec_kw = {'width_ratios':[1, 1, 1.2]})
cmap = mpl.cm.nipy_spectral

#cmap = sns.color_palette("Spectral", as_cmap=True)

bin1 = aes.plot_corr_dist_axes(axes[0], Forces['ANI'], Forces['DFT'], cmap, r'ANI-1x Force ($kcal \times mol^{-1} \times \AA^{-1}$)', r'DFT Force ($kcal \times mol^{-1} \times \AA^{-1}$)','ANI-1x')
heights = bin1[0]
bin2 = aes.plot_corr_dist_axes(axes[1], Forces['DFTB'], Forces['DFT'], cmap, r'DFTB Force ($kcal \times mol^{-1} \times \AA^{-1}$)', '','DFTB', 1, heights.max())
bin3 = aes.plot_corr_dist_axes(axes[2], Forces['PM6'], Forces['DFT'], cmap, r'PM6 Force ($kcal \times mol^{-1} \times \AA^{-1}$)', '','PM6', 1, heights.max())

#bin1 = aes.plot_corr_dist_axes(axes[0], forces['ANI']['TrpCage1L2Y'], forces['DFT']['TrpCage1L2Y'], cmap, r'ANI-1x Force ($kcal \times mol^{-1} \times \AA^{-1}$)', r'DFT Force ($kcal \times mol^{-1} \times \AA^{-1}$)')
#heights = bin1[0]
#bin2 = aes.plot_corr_dist_axes(axes[1], forces['DFTB']['TrpCage1L2Y'], forces['DFT']['TrpCage1L2Y'], cmap, r'DFTB Force ($kcal \times mol^{-1} \times \AA^{-1}$)', '', 1, heights.max())
#bin3 = aes.plot_corr_dist_axes(axes[2], forces['PM6']['TrpCage1L2Y'], forces['DFT']['TrpCage1L2Y'], cmap, r'PM6 Force ($kcal \times mol^{-1} \times \AA^{-1}$)', '', 1, heights.max())

cb1 = fig.colorbar(bin2[-1], cmap=cmap)
cb1.set_label('Count', fontsize=26)
cb1.ax.tick_params(labelsize=30) 
#aes.add_inset_histogram(Forces['ANI'], Forces['DFT'], pos=[0.15, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#aes.add_inset_histogram(Forces['DFTB'], Forces['DFT'], pos=[0.41, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#aes.add_inset_histogram(Forces['PM6'], Forces['DFT'], pos=[0.67, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#add_inset_histogram(ax, Xa, Xp)
#add_inset_histogram(ax, Xa, Xp)

#plt.show()
pp = PdfPages(storepath)
pp.savefig(fig)
pp.close()

FileNotFoundError: [Errno 2] No such file or directory: '/home/jujuman/Dropbox/ANAKIN-ME-RESEARCH/PUBLICATION-ActiveLearning/Figures/force_ANIMD_compare_complete.pdf'

In [ ]:
storepath = '/home/jujuman/Dropbox/ANAKIN-ME-RESEARCH/PUBLICATION-ActiveLearning/Figures/denergy_ANIMD_compare.pdf'

label_size = 20
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size

#'$F_{dft}$' + r' $(kcal \times mol^{-1} \times \AA^{-1})$'

fig, axes = plt.subplots(1, 1, figsize=[10,8])
cmap = mpl.cm.viridis

bin1 = aes.plot_corr_dist_axes(axes, forces['ANI']['Atazanavir'], forces['DFT']['Atazanavir'], cmap, r'ANI-1x $\Delta$E ($kcal \times mol^{-1}$)', r'DFT $\Delta$E ($kcal \times mol^{-1}$)','ANI-1x')

cb1 = fig.colorbar(bin1[-1], cmap=cmap)
cb1.set_label('Count', fontsize=20)

#aes.add_inset_histogram(Forces['ANI'], Forces['DFT'], pos=[0.15, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#aes.add_inset_histogram(Forces['DFTB'], Forces['DFT'], pos=[0.41, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#aes.add_inset_histogram(Forces['PM6'], Forces['DFT'], pos=[0.67, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#add_inset_histogram(ax, Xa, Xp)
#add_inset_histogram(ax, Xa, Xp)

plt.show()
#pp = PdfPages(storepath)
#pp.savefig(fig)
#pp.close()

In [ ]:
plt.hist(Forces['DFTB'] - Forces['DFT'],bins=100)
plt.hist(Forces['ANI'] - Forces['DFT'],bins=100, alpha=0.5)

plt.show()

In [ ]:
Ed = forces['DFT']['Retinol']
Ea = forces['ANI']['Retinol']

fig = plt.figure(figsize=[8,8])
plt.scatter(Ed, Ea)
plt.plot([Ed.min(), Ed.max()], [Ed.min(), Ed.max()],'--',c='r')
plt.xlim([Ed.min(), Ed.max()])
plt.ylim([Ed.min(), Ed.max()])
plt.xlabel(r"E DFT ($kcal \times mol^{-1}$)", fontsize=18)
plt.ylabel(r"E ani ($kcal \times mol^{-1}$)", fontsize=18)
plt.show()
#pp = PdfPages("/home/jujuman/Dropbox/ANAKIN-ME-RESEARCH/Figures/NEW-11-30-17/fent_ecorr_plot.pdf")
#pp.savefig(fig)
#pp.close()